In [44]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
import concurrent.futures
from logzero import logger

In [60]:
User_Master = []
# Base User class
class User:
    def __init__(self, name, broker, **kwargs):

        # The User class initializes with name, broker, and other attributes that vary depending on the broker.

        self.name = name
        self.broker = broker

        # Set dynamic attributes based on the broker
        self.username = kwargs.get("username", None)
        self.api_key = kwargs.get("api_key", None)
        self.pwd = kwargs.get("pwd", None)
        self.totp_token = kwargs.get("totp_token", None)
        self.api_obj = None  # Placeholder for api obj
        self.auth_token = None  # Placeholder for login access token
        self.refresh_token = None  # Placeholder for login refresh token
        self.feed_token = None  # Placeholder for login feed token

        # Add user to master
        User_Master.append(self)

        # Call the login method automatically when the object is created
        print(self.user_login())

    def get_user_info(self):

        # Returns the user information in a formatted string.
        return f"Name: {self.name}, Broker: {self.broker}, Username: {self.username}"

 # Login Functions
    def user_login(self):

        print(f"Logging in {self.name}:{self.username} to {self.broker}...")

        # Handling Multiple Brokers

        if self.broker == "AngelOne":
            return self._login_angel_one()
        elif self.broker == "OtherBroker":
            return self._login_other_broker()
        else:
            return "Unsupported broker. Please provide valid credentials."

    def _login_angel_one(self):

        # Login using Angel One's SmartAPI and generates an access token.

        try:
            token = self.totp_token
            totp = pyotp.TOTP(token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e

        try:
            # SmartAPI login for Angel One
            self.api_obj = SmartConnect(api_key=self.api_key)

            data = self.api_obj.generateSession(self.username, self.pwd, totp)

            self.auth_token = data['data']['jwtToken']          # Store access token
            self.refresh_token = data['data']['refreshToken']   # Store refresh_token
            self.feed_token = self.api_obj.getfeedToken()       # Store Feed token

            return f"{self.api_obj.getProfile(self.refresh_token)["message"]}"

        except Exception as e:
            return f"Angel One login failed: {str(e)}"

    def _login_other_broker(self):
        """
        Placeholder login function for other brokers.
        """

 # Logout Functions
    def user_logout(self):

        # Simulates user logout

        print(f"Logging out {self.name} to {self.broker}...")

        if self.broker == "AngelOne":
            return self._logout_angel_one()
        elif self.broker == "OtherBroker":
            return self._logout_other_broker()
        else:
            return "Logout Failed."

    def _logout_angel_one(self):

        # Logout using Angel One's SmartAPI.

        try:
            # SmartAPI logout for Angel One
            res = self.api_obj.terminateSession(self.username)

            self.auth_token = None       # Reset access token
            self.refresh_token = None    # Reset refresh_token
            self.feed_token = None       # Reset Feed token

            return f"{res["data"]}"

        except Exception as e:
            return f"Angel One logout failed: {str(e)}"

    def _logout_other_broker(self):
        """
        Placeholder logout function for other brokers.
        """

 # Order Placing function

    def place_order(self, order_details):

        #  Handling Multiple Brokers

        if self.broker == "AngelOne":
            return self._place_order_angel_one(order_details)
        elif self.broker == "OtherBroker":
            return self._place_order_other_broker(order_details)
        else:
            return "Unsupported broker."

    def _place_order_angel_one(self, order_details):
        # placing order using Angel One's SmartAPI
        try:
            # # Method 1: Place an order and return the order ID
            # res = self.api_obj.placeOrder(order_details)

            # Method 2: Place an order and return the full response
            res = self.api_obj.placeOrderFullResponse(order_details)
            return f" {res['message']} , Details : {res['data']}"

        except Exception as e:
            return f"Order placement failed: {e}"

    def _place_order_other_broker(self, order_details):
        """
        Placeholder order function for other brokers.
        """

# Parent User class with broker-specific attributes and children management
class ParentUser(User):
    def __init__(self, name, broker, **kwargs):

        # ParentUser class inherits from User and can manage child users.

        super().__init__(name, broker, **kwargs)
        self.children = []

    def add_child(self, child):

        # Adds a ChildUser to the ParentUser's list of children.

        self.children.append(child)

    def list_children(self):

        # Returns a list of the usernames of all children under the ParentUser.

        return [child.name for child in self.children]

    def place_order_for_child(self, order_details):

        #list to store results
        Child_order_results = []

        # ThreadPoolExecutor to place orders for for all children concurrently
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(child.place_order, order_details): child for child in self.children}

            for future in concurrent.futures.as_completed(futures):
                child = futures[future]
                try:
                    result = future.result()  # Get the result of place_order()
                    Child_order_results.append((child.get_user_info(), result))  # Save user and result
                except Exception as e:
                    print(f"Child User: {child.get_user_info()} generated an exception: {e}")

        # Print order results
        for child, result in Child_order_results:
            print(f"Child User: {child}, Order Status: {result}")

    def place_order_for_all(self, order_details):

        # Places an order for both the parent and all children.
        res = self.place_order(order_details)
        print(f"Parent User: {self.get_user_info()}, Order Status:{res}")
        self.place_order_for_child(order_details)

# Child User class (must be linked to a ParentUser)
class ChildUser(User):
    def __init__(self, name, broker, parent, **kwargs):

        # ChildUser class inherits from User and is linked to a ParentUser.

        super().__init__(name, broker, **kwargs)
        self.parent = parent
        parent.add_child(self)  # Automatically add the child to the parent's list

    def get_parent_info(self):

        # Returns the parent user's information.

        return f"Parent Name: {self.parent.name}, Parent Broker: {self.parent.broker}"

def master_login(User_Master):

      # List to store the results
      login_results = []

      # Using ThreadPoolExecutor to run user_login for each user concurrently
      with concurrent.futures.ThreadPoolExecutor() as executor:
            # Submit each task (user.user_login()) and store the future objects
            futures = {executor.submit(user.user_login): user for user in User_Master}

            # As each future completes, retrieve its result and store it in the results list
            for future in concurrent.futures.as_completed(futures):
                  user = futures[future]  # Get the user associated with this future
                  try:
                        result = future.result()  # Get the result of user_login()
                        login_results.append((user.get_user_info(), result))  # Save user and result
                  except Exception as e:
                        print(f"{user} generated an exception: {e}")

      # Print or use the results
      for user, result in login_results:
            print(f"User: {user}, Login result: {result}")

def master_logout(User_Master):
      # Create a list to store the results
      logout_results = []

      # Using ThreadPoolExecutor to run user_logout for each user concurrently
      with concurrent.futures.ThreadPoolExecutor() as executor:
            # Submit each task (user.user_logout()) and store the future objects
            futures = {executor.submit(user.user_logout): user for user in User_Master}

            # As each future completes, retrieve its result and store it in the results list
            for future in concurrent.futures.as_completed(futures):
                  user = futures[future]  # Get the user associated with this future
                  try:
                        result = future.result()  # Get the result of user_logout()
                        logout_results.append((user.get_user_info(), result))  # Save user and result
                  except Exception as exc:
                        print(f"{user} generated an exception: {exc}")

      # Print or use the results
      for user, result in logout_results:
            print(f"User: {user}, Logout result: {result}")

In [55]:
# Example Usage
# Creating a ParentUser with custom attributes based on the broker
parent = ParentUser(name="Yash",
                    broker="AngelOne",
                    api_key = 'bE7ipU6q',
                    username = 'Y58983127',
                    pwd = '1416',
                    totp_token = "NQST6OLTU6MP3PKRH2BLXUTGDI")

# Creating ChildUser instances and linking them to the parent
child1 = ChildUser( name="Ravi",
                    broker="AngelOne",
                    parent=parent,
                    api_key = 'eskMiA2H',
                    username = 'R947822',
                    pwd = '1234',
                    totp_token = "E4FCOCHUNVRDG7ANFXK6D3OQVA")

Logging in Yash:Y58983127 to AngelOne...


[I 241023 01:51:53 smartConnect:121] in pool


SUCCESS
Logging in Ravi:R947822 to AngelOne...


[I 241023 01:51:57 smartConnect:121] in pool


SUCCESS


In [49]:
# Print user info
print(User_Master)
print(child1.get_parent_info())       # Parent user information
print(parent.list_children())         # Children user information

[<__main__.ParentUser object at 0x000001B7E1C79100>, <__main__.ChildUser object at 0x000001B7E172A690>]
Parent Name: Yash, Parent Broker: AngelOne
['Ravi']


In [50]:
# login in all users
master_login(User_Master)

Logging in Yash:Y58983127 to AngelOne...
Logging in Ravi:R947822 to AngelOne...


[I 241023 01:37:16 smartConnect:121] in pool
[I 241023 01:37:16 smartConnect:121] in pool


User: Name: Ravi, Broker: AngelOne, Username: R947822,, Login result: SUCCESS
User: Name: Yash, Broker: AngelOne, Username: Y58983127,, Login result: SUCCESS


In [56]:
# Placing an order for both parent and children
order_details = {
                "variety": "NORMAL",
                "tradingsymbol": "SBIN-EQ",
                "symboltoken": "3045",
                "transactiontype": "BUY",
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0",
                "squareoff": "0",
                "stoploss": "0",
                "quantity": "1"
                }

parent.place_order_for_all(order_details=order_details)


Parent User: Name: Yash, Broker: AngelOne, Username: Y58983127, Order Status: SUCCESS , Details : {'script': 'SBIN-EQ', 'orderid': '241023100001225', 'uniqueorderid': 'aab9d1a7-5789-4b57-a9d1-8958100ecdba'}
Child User: Name: Ravi, Broker: AngelOne, Username: R947822, Order Status:  SUCCESS , Details : {'script': 'SBIN-EQ', 'orderid': '241023000000418', 'uniqueorderid': '9a6270eb-ceb3-46cc-afd1-b47f997c53b3'}


In [57]:
parent.api_obj.individual_order_details('aab9d1a7-5789-4b57-a9d1-8958100ecdba')

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': {'variety': 'NORMAL',
  'ordertype': 'MARKET',
  'producttype': 'INTRADAY',
  'duration': 'DAY',
  'price': 0.0,
  'triggerprice': 0.0,
  'quantity': '1',
  'disclosedquantity': '0',
  'squareoff': 0.0,
  'stoploss': 0.0,
  'trailingstoploss': 0.0,
  'tradingsymbol': 'SBIN-EQ',
  'transactiontype': 'BUY',
  'exchange': 'NSE',
  'symboltoken': '3045',
  'instrumenttype': '',
  'strikeprice': -1.0,
  'optiontype': '',
  'expirydate': '',
  'lotsize': '1',
  'cancelsize': '0',
  'averageprice': 0.0,
  'filledshares': '0',
  'unfilledshares': '1',
  'orderid': '',
  'text': '',
  'status': 'open',
  'orderstatus': 'open',
  'updatetime': '',
  'exchtime': '',
  'exchorderupdatetime': '',
  'fillid': '',
  'filltime': '',
  'parentorderid': '',
  'ordertag': '',
  'uniqueorderid': 'aab9d1a7-5789-4b57-a9d1-8958100ecdba'}}

In [58]:
child1.api_obj.individual_order_details('9a6270eb-ceb3-46cc-afd1-b47f997c53b3')

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': {'variety': 'NORMAL',
  'ordertype': 'MARKET',
  'producttype': 'INTRADAY',
  'duration': 'DAY',
  'price': 0.0,
  'triggerprice': 0.0,
  'quantity': '1',
  'disclosedquantity': '0',
  'squareoff': 0.0,
  'stoploss': 0.0,
  'trailingstoploss': 0.0,
  'tradingsymbol': 'SBIN-EQ',
  'transactiontype': 'BUY',
  'exchange': 'NSE',
  'symboltoken': '3045',
  'instrumenttype': '',
  'strikeprice': -1.0,
  'optiontype': '',
  'expirydate': '',
  'lotsize': '1',
  'cancelsize': '0',
  'averageprice': 0.0,
  'filledshares': '0',
  'unfilledshares': '1',
  'orderid': '',
  'text': '',
  'status': 'open',
  'orderstatus': 'open',
  'updatetime': '',
  'exchtime': '',
  'exchorderupdatetime': '',
  'fillid': '',
  'filltime': '',
  'parentorderid': '',
  'ordertag': '',
  'uniqueorderid': '9a6270eb-ceb3-46cc-afd1-b47f997c53b3'}}

In [59]:
master_logout(User_Master)

Logging out Yash to AngelOne...Logging out Ravi to AngelOne...

User: Name: Ravi, Broker: AngelOne, Username: R947822, Logout result: Logout Successfully
User: Name: Yash, Broker: AngelOne, Username: Y58983127, Logout result: Logout Successfully
